In [1]:
import numpy as np
import numpy.linalg
import scipy.stats
import MDAnalysis as mda
import mdtraj as md
import MDAnalysis.lib.distances
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
import MDAnalysis.lib.NeighborSearch as NS
from scipy.spatial import Voronoi, voronoi_plot_2d, SphericalVoronoi, Delaunay
import multiprocessing as mp
import pandas as pd
import seaborn as sns

In [2]:
ref_structure="ALK1_50_CG3_01_npt_eq.pdb"
traj         ="ALK1_50_CG3_01_protein_final.xtc"
Name         ='ALK1_50_CG3_01'
u            = mda.Universe(ref_structure,traj)
ref          = mda.Universe(ref_structure)
    #frames       = len(u.trajectory)
    
topology     = md.load(ref_structure).topology
trajectory   = md.load(traj, top=ref_structure)
frames       = trajectory.n_frames #Number of frames
chains       = topology.n_chains   #Number of chains
atoms        = int(topology.n_atoms/chains)#Number of atoms in each monomer 
AminoAcids   = int(topology.n_residues/chains) #Number of residues per chain 
    
res          = 'LYS'    
sel1         = 'resname %s and name BB'%res    

/home/yadav129/anaconda3/lib/python3.9/site-packages/MDAnalysis/topology/PDBParser.py:317: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


RES_Sel   =ref.select_atoms('bynum '+ str(0) + ':' + str(atoms) +' and name BB' +' and resname LYS').center_of_mass()
#num_res   =len(list(RES_Sel.residues.resnames))

RES_Sel

def GetCoordsCOM(ts):
    
    points=[]
    RES_Sel   =ref.select_atoms('bynum '+ str(0) + ':' + str(atoms) +' and name BB' +' and resname LYS').center_of_mass()
    #num_res   =len(list(RES_Sel.residues.resnames))

    atoms3    = u.select_atoms(sel1)
    chain1    = []
    chain2    = []
    #print (sel1.positions)
    

    atom_Groups = [[] for x in range(chains)]
    m_start     = 0
    for m in range(0,chains):
        m_end   = atoms * (m+1)
        atom_Groups[m].extend([u.select_atoms('bynum '+ str(m_start) + ':' + str(m_end) +' and name BB' +' and resname LYS')]) 
        m_start = m_end + 1
        mySet   = set([])   
         
        #print (atom_Groups[m][0].center_of_mass())
        points.append(atom_Groups[m][0].center_of_mass())
    return points
       

In [3]:
def GetCoordsCOM(ts):
    timestep=200
    
    points = []
    
    RES_Sel   =ref.select_atoms('bynum '+ str(0) + ':' + str(atoms) +' and name BB' +' and resname LYS')
    num_res   =len(list(RES_Sel.residues.resnames))

    atoms3    = u.select_atoms(sel1)
    chain1    = []
    chain2    = []
    #print (sel1.positions)
    
    for tt in tqdm(u.trajectory[::timestep]):
        atom_Groups = [[] for x in range(chains)]
        m_start     = 0
        for m in range(0,chains):
            m_end   = atoms * (m+1)
            atom_Groups[m].extend([u.select_atoms('bynum '+ str(m_start) + ':' + str(m_end) +' and name BB' +' and resname LYS')]) 
            m_start = m_end + 1
            mySet   = set([])   
            
            
            points.append(atom_Groups[m][0].center_of_mass())
    return points

In [4]:
def EdgeLength(A,B):
    dist=math.sqrt( ((float(A[0]) -float(B[0]))**2 ) +  ((float(A[1]) -float(B[1]))**2 ) + ((float(A[2]) -float(B[2]))**2 )   )
    return dist

In [5]:
def Delaunay_Tessellation(ts):
    print ('Frame %i in %i \n'%(ts, end_f))
    points = GetCoordsCOM(ts)
    tri    = Delaunay(points)   
    
    #plt.plot(points[:, 0], points[:, 1], points[:, 2], c='b', linestyle='dashed')
    #plt.plot(sv.vertices[:, 0], sv.vertices[:, 1], sv.vertices[:, 2], c='g')
    #plt.show()
    #print ("shape", tri.simplices.shape)
    #print ("#",tri.simplices)
    #tri.simplices[0]
    #print ("#",tri.neighbors)
    #points[tri.simplices]

    ridge_points   = tri.simplices
    r_length       = len(ridge_points)
    tetrahedrality = []
    for k in range(0, r_length):
        
        sum_edge       = 0
        sumd           = 0
        tetra          = 0
        lbar           = 0
        ridge_k = ridge_points[k]
        L0 = points[int(ridge_k[0])]
        L1 = points[int(ridge_k[1])] 
        L2 = points[int(ridge_k[2])] 
        L3 = points[int(ridge_k[3])]   
        #print (ridge_k, L0, L1, L2, L3) 
        
        L01 = EdgeLength(L0,L1)
        L02 = EdgeLength(L0,L2)
        L03 = EdgeLength(L0,L3)
        L12 = EdgeLength(L1,L2)
        L13 = EdgeLength(L1,L3)
        L23 = EdgeLength(L2,L3)
        
        sumd += L01 + L02 + L03 + L12 + L13 + L23 
        lbar= sumd/15.0
        
        a1  = EdgeLength(L0,L1) - EdgeLength(L0,L2)
        a2  = EdgeLength(L0,L1) - EdgeLength(L0,L3)
        a3  = EdgeLength(L0,L1) - EdgeLength(L1,L2)
        a4  = EdgeLength(L0,L1) - EdgeLength(L1,L3)
        a5  = EdgeLength(L0,L1) - EdgeLength(L2,L3)
        
        a6  = EdgeLength(L0,L2) - EdgeLength(L0,L3)
        a7  = EdgeLength(L0,L2) - EdgeLength(L1,L2)
        a8  = EdgeLength(L0,L2) - EdgeLength(L1,L3)
        a9  = EdgeLength(L0,L2) - EdgeLength(L2,L3)
        
        a10 = EdgeLength(L0,L3) - EdgeLength(L1,L2)
        a11 = EdgeLength(L0,L3) - EdgeLength(L1,L2)
        a12 = EdgeLength(L0,L3) - EdgeLength(L1,L2)
        
        a13 = EdgeLength(L1,L2) - EdgeLength(L1,L3)
        a14 = EdgeLength(L1,L2) - EdgeLength(L2,L3)
        
        a15 = EdgeLength(L1,L3) - EdgeLength(L2,L3)
        
        
        sum_edge += a1*a1 + a2*a2 + a3*a3 + a4*a4 + a5*a5 + a6*a6 + a7*a7 + a8*a8 + a9*a9 + a10*a10  + a11*a11 + a12*a12 + a13*a13 + a14*a14 + a15*a15    
        
        tetra = sum_edge/(15*lbar*lbar) 
        #print ("lbar", lbar, "sumd", sumd, "sum_edge", sum_edge, "tet", tetra)
        tetrahedrality.append(tetra) 
        
    return (tetrahedrality)

In [6]:
def Histogram(data):
    mat   = np.asarray(pd.DataFrame(data)).flatten()
    mat1  = mat[np.logical_not(np.isnan(mat))]
    
    np.savetxt(Name+'_Tetrahedrality.dat', mat1, header=Name)
    
    ax    = sns.distplot(mat1, hist=True, kde=True, color="r", label=Name, hist_kws={ "alpha":1, "linewidth": 3})
    ax.set_ylabel("Probability")
    ax.set_xlabel("T")
    ax.set_title("Tetrahedrality", fontsize='large')
    ax.legend()
    plt.savefig(Name+"_Tetrahedrality.png", dpi=600, facecolor='w', edgecolor='w',orientation='portrait')   
    #plt.show()

In [ ]:
if __name__ == "__main__":
   
    ref_structure="ALK1_50_CG3_01_npt_eq.pdb"
    traj         ="ALK1_50_CG3_01_protein_final.xtc"
    Name         ='ALK1_50_CG3_01_xx'
    
    u            = mda.Universe(ref_structure,traj)
    ref          = mda.Universe(ref_structure)
    #frames       = len(u.trajectory)
    
    topology     = md.load(ref_structure).topology
    trajectory   = md.load(traj, top=ref_structure)
    frames       = trajectory.n_frames #Number of frames
    chains       = topology.n_chains   #Number of chains
    atoms        = int(topology.n_atoms/chains)#Number of atoms in each monomer 
    AminoAcids   = int(topology.n_residues/chains) #Number of residues per chain 
    
    res          = 'LYS'    
    sel1         = 'resname %s and name BB'%res

    print ("There are {} chains and each has {} number of AminoAcids".format(AminoAcids,chains))
    
    
    start_f      = 0
    skip         = 200
    end_f        = frames  
    nprocs       = 4
    n_frames     = np.arange(start_f, end_f)[::skip]
    pool         = mp.Pool(processes=nprocs)
    
    
    print ('Initiating multiprocessing with %i processors'%nprocs)
    
    Tetra        = pool.map(Delaunay_Tessellation,n_frames) 
    Histogram(Tetra)
    
    #GetCoords(n_frames)
    

There are 16 chains and each has 50 number of AminoAcids
Frame 0 in 20001 
Frame 1400 in 20001 
Frame 4200 in 20001 
Frame 2800 in 20001 





  0%|                                                                                                                    | 0/101 [00:00<?, ?it/s]

  0%|                                                                                                                    | 0/101 [00:00<?, ?it/s]

Initiating multiprocessing with 4 processors


 32%|█████████████████████████████████▉                                                                         | 32/101 [00:01<00:02, 32.17it/s]malloc(): invalid size (unsorted)
free(): invalid next size (normal)
 36%|██████████████████████████████████████▏                                                                    | 36/101 [00:01<00:01, 32.75it/s]

...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:02<00:00, 36.12it/s]


Frame 18000 in 20001 



  0%|                                                                                                                                                    | 0/101 [00:00<?, ?it/s]

Frame 19400 in 20001 



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:02<00:00, 35.45it/s]
